In [2]:
from database_manager import DatabaseManager

dm = DatabaseManager()


dm.check_all_tables_data()

'rag' 데이터베이스가 준비되었습니다.

===== 모든 테이블 데이터 조회 시작 =====
테이블에 데이터가 없습니다.
---------------------------------

연결이 종료되었습니다.
테이블에 데이터가 없습니다.
---------------------------------

연결이 종료되었습니다.
테이블에 데이터가 없습니다.
---------------------------------

연결이 종료되었습니다.
테이블에 데이터가 없습니다.
---------------------------------

연결이 종료되었습니다.
테이블에 데이터가 없습니다.
---------------------------------

연결이 종료되었습니다.

===== 모든 테이블 데이터 조회 완료 =====


In [ ]:
# pyqt 디자인 해보기
# rag 성능개선은 어떻게?

In [ ]:
import sys
import requests
from PyQt6.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QLabel,
    QLineEdit, QTextEdit, QFrame, QStackedWidget, QListWidget, QListWidgetItem, QFileDialog
)
from PyQt6.QtCore import Qt, QSize
import os

class LoginWidget(QWidget):
    """로그인 및 회원가입 UI를 담당하는 위젯"""
    def __init__(self, parent=None):
        super().__init__(parent)
        self.parent_app = parent
        layout = QVBoxLayout()
        layout.setSpacing(15)
        layout.setAlignment(Qt.AlignmentFlag.AlignCenter)

        title = QLabel("PDF RAG")
        title.setStyleSheet("font-size: 24px; font-weight: bold;")
        
        self.email_input = QLineEdit()
        self.email_input.setPlaceholderText("이메일")
        self.password_input = QLineEdit()
        self.password_input.setPlaceholderText("비밀번호")
        self.password_input.setEchoMode(QLineEdit.EchoMode.Password)

        self.login_button = QPushButton("로그인")
        self.signup_button = QPushButton("회원가입")
        
        self.status_label = QLabel("")
        self.status_label.setStyleSheet("color: red;")

        layout.addWidget(title, alignment=Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(QLabel("이메일:"))
        layout.addWidget(self.email_input)
        layout.addWidget(QLabel("비밀번호:"))
        layout.addWidget(self.password_input)
        layout.addWidget(self.login_button)
        layout.addWidget(self.signup_button)
        layout.addWidget(self.status_label, alignment=Qt.AlignmentFlag.AlignCenter)

        self.login_button.clicked.connect(self.login)
        self.signup_button.clicked.connect(self.signup)
        
        self.setLayout(layout)

    def login(self):
        email = self.email_input.text()
        password = self.password_input.text()
        self.parent_app.login(email, password)
    
    def signup(self):
        email = self.email_input.text()
        password = self.password_input.text()
        self.parent_app.signup(email, password)

class MainAppWidget(QWidget):
    """로그인 후의 메인 애플리케이션 UI를 담당하는 위젯"""
    def __init__(self, parent=None):
        super().__init__(parent)
        self.parent_app = parent
        main_layout = QHBoxLayout()

        # 왼쪽: 세션 목록
        left_panel = QVBoxLayout()
        left_panel.addWidget(QLabel("채팅 세션 목록"))
        self.session_list = QListWidget()
        self.new_chat_button = QPushButton("새로운 PDF로 채팅 시작")
        left_panel.addWidget(self.session_list)
        left_panel.addWidget(self.new_chat_button)
        
        # 오른쪽: 채팅 화면
        right_panel = QVBoxLayout()
        self.chat_display = QTextEdit()
        self.chat_display.setReadOnly(True)
        self.question_input = QLineEdit()
        self.question_input.setPlaceholderText("질문을 입력하세요...")
        self.send_button = QPushButton("전송")
        
        question_layout = QHBoxLayout()
        question_layout.addWidget(self.question_input)
        question_layout.addWidget(self.send_button)
        
        right_panel.addWidget(self.chat_display)
        right_panel.addLayout(question_layout)

        main_layout.addLayout(left_panel, 1) # 1:3 비율
        main_layout.addLayout(right_panel, 3)

        self.new_chat_button.clicked.connect(self.parent_app.start_new_session)
        self.send_button.clicked.connect(self.parent_app.ask_question)
        self.session_list.currentItemChanged.connect(self.parent_app.session_selected)
        
        self.setLayout(main_layout)

class RAGClientApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.server_url = "http://127.0.0.1:5000"
        self.user_id = None
        self.current_session_id = None
        
        self.setWindowTitle("PDF 기반 RAG 시스템")
        self.setGeometry(200, 200, 800, 600)
        
        self.stacked_widget = QStackedWidget()
        self.setCentralWidget(self.stacked_widget)
        
        self.login_widget = LoginWidget(self)
        self.main_app_widget = MainAppWidget(self)
        
        self.stacked_widget.addWidget(self.login_widget)
        self.stacked_widget.addWidget(self.main_app_widget)

    def login(self, email, password):
        try:
            response = requests.post(
                f"{self.server_url}/login",
                data={"username": email, "password": password}
            )
            if response.status_code == 200:
                self.user_id = response.json()['user_id']
                self.stacked_widget.setCurrentWidget(self.main_app_widget)
                # self.load_sessions() # 로그인 후 세션 목록 로드 (서버에 /sessions/list 엔드포인트 구현 필요)
            else:
                self.login_widget.status_label.setText(response.json().get('detail', '로그인 실패'))
        except requests.RequestException as e:
            self.login_widget.status_label.setText(f"서버 연결 오류: {e}")

    def signup(self, email, password):
        try:
            response = requests.post(
                f"{self.server_url}/signup",
                json={"email": email, "password": password}
            )
            if response.status_code == 200:
                self.login_widget.status_label.setText("회원가입 성공! 이제 로그인하세요.")
            else:
                self.login_widget.status_label.setText(response.json().get('detail', '회원가입 실패'))
        except requests.RequestException as e:
            self.login_widget.status_label.setText(f"서버 연결 오류: {e}")

    def start_new_session(self):
        file_path, _ = QFileDialog.getOpenFileName(self, "PDF 파일 선택", "", "PDF Files (*.pdf)")
        if file_path:
            self.upload_and_create_session(file_path)
            
    def upload_and_create_session(self, file_path):
        filename = os.path.basename(file_path)
        self.main_app_widget.chat_display.setText(f"'{filename}' 업로드 및 처리 중...")
        QApplication.processEvents()
        
        try:
            with open(file_path, 'rb') as f:
                files = {'file': (filename, f, 'application/pdf')}
                response = requests.post(
                    f"{self.server_url}/sessions/create_with_pdf?user_id={self.user_id}", 
                    files=files, 
                    timeout=300
                )
            
            response.raise_for_status()
            data = response.json()
            
            # 새 세션을 목록에 추가
            item = QListWidgetItem(data['session_title'])
            item.setData(Qt.ItemDataRole.UserRole, data['session_id']) # 데이터 저장
            self.main_app_widget.session_list.addItem(item)
            self.main_app_widget.session_list.setCurrentItem(item)

        except requests.RequestException as e:
            self.main_app_widget.chat_display.setText(f"세션 생성 실패: {e}")

    def session_selected(self, current, previous):
        if current:
            self.current_session_id = current.data(Qt.ItemDataRole.UserRole)
            self.main_app_widget.chat_display.setText(f"'{current.text()}' 세션이 선택되었습니다.\n질문을 입력하세요.")
            # 여기에 이전 대화 기록을 불러오는 로직을 추가할 수 있습니다.
            # 예: self.load_chat_history(self.current_session_id)

    def ask_question(self):
        if not self.current_session_id:
            self.main_app_widget.chat_display.append("\n[오류] 먼저 세션을 선택하거나 생성해주세요.")
            return
            
        question = self.main_app_widget.question_input.text().strip()
        if not question:
            return
            
        self.main_app_widget.chat_display.append(f"\n나: {question}")
        self.main_app_widget.question_input.clear()
        QApplication.processEvents()

        try:
            payload = {"question": question, "session_id": self.current_session_id}
            response = requests.post(f"{self.server_url}/ask", json=payload, timeout=60)
            response.raise_for_status()
            answer = response.json().get("answer", "답변 없음")
            self.main_app_widget.chat_display.append(f"\n봇: {answer}")
        except requests.RequestException as e:
            self.main_app_widget.chat_display.append(f"\n[오류] 서버 통신 실패: {e}")

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = RAGClientApp()
    ex.show()
    sys.exit(app.exec())


C:\Users\minja\AppData\Local\Temp\ipykernel_14548\3930552649.py:99: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  super().__init__()
C:\Users\minja\AppData\Local\Temp\ipykernel_14548\3930552649.py:13: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  super().__init__(parent)
C:\Users\minja\AppData\Local\Temp\ipykernel_14548\3930552649.py:61: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  super().__init__(parent)


SystemExit: 0

c:\Users\minja\anaconda3\envs\rag\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
